## <div class="alert alert-block alert-success"><center>Data Preparation - Obama Part<center>

In [1]:
%pylab inline
import pandas as pd
df_obama2=pd.read_csv('obama2012.csv')

Populating the interactive namespace from numpy and matplotlib


/Users/zhanghuishan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (10,11,14,64) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
len(df_obama2)

118455

In [3]:
df_obama2 = df_obama2[['committee_id', 'report_year', 'disbursement_date','recipient_name',\
                     'recipient_state','recipient_city',\
                     'disbursement_purpose_category','disbursement_description','disbursement_amount']]

In [4]:
df_obama2.head() 

,committee_id,report_year,disbursement_date,recipient_name,recipient_state,recipient_city,disbursement_purpose_category,disbursement_description,disbursement_amount
0,C00431445,2012,12/31/12 0:00,"ZISMAN, PARVIN A.",NY,NEW YORK,REFUNDS,CHARGEBACK,25.00
1,C00431445,2012,12/31/12 0:00,"ZEHNER, CORNELIA G MS.",TX,DALLAS,OTHER,DISGORGED TO US TREASURY,360.87
2,C00431445,2012,12/31/12 0:00,"ZEHNER, CORNELIA G MS.",TX,DALLAS,OTHER,CONTRIBUTION REFUND,-360.87
3,C00431445,2012,12/31/12 0:00,"YOUNGSTEIN, KENNETH",NaN,SWITZERLAND,OTHER,CONTRIBUTION REFUND,250.00
4,C00431445,2012,12/31/12 0:00,"YEAGER, BETH ANNE MS",WI,MADISON,REFUNDS,VOIDED CHECK,-200.00


In [2]:
df_obama2.drop(df_obama2[df_obama2['disbursement_purpose_category'] == 'REFUNDS'].index, inplace = True) 
df_obama2.drop(df_obama2[df_obama2['disbursement_description'] == 'CONTRIBUTION REFUND'].index, inplace = True) 
df_obama2 = df_obama2[df_obama2['disbursement_description'].notna()]

In [6]:
df_obama2.isnull().sum()

committee_id                      0
report_year                       0
disbursement_date                 0
recipient_name                    0
recipient_state                  45
recipient_city                   20
disbursement_purpose_category     0
disbursement_description          0
disbursement_amount               0
dtype: int64

In [7]:
df_obama2.disbursement_description.nunique()

114

In [8]:
df_obama2.disbursement_description.unique()

array(['DISGORGED TO US TREASURY', 'CHARGEBACK', 'VOIDED CHECK',
       'CREDIT CARD FEES', 'PAYROLL TAXES', 'UTILITIES', 'RENT/OCCUPANCY',
       'EQUIPMENT MAINTENANCE', 'DISGORGEMENT', 'LICENSE FEES',
       'INSURANCE', 'TRAVEL/LODGING', 'PARKING', 'OFFICE SUPPLIES',
       'PRINTING', 'POSTAGE', 'CATERING/FACILITIES', 'DELIVERY',
       'MEDIA BUY', 'INTERPRETING SERVICE', 'EQUIPMENT RENTAL',
       'EVENT SITE RENTAL', 'TELEMARKETING', 'TELEPHONE',
       'WEB SITE HOSTING', 'TECHNOLOGY CONSULTING',
       'STAGING, SOUND, LIGHTING', 'EVENT PLANNING SERV.', 'PAYROLL',
       'REVENUE PROCESSING FEES', 'COMPUTER EQUIPMENT',
       'PAYROLL PROCESSING FEE', 'MEDIA PRODUCTION',
       'CONS./PROF.SERV. MEDIA', 'STRATEGIC CONSULTING SERVICES',
       'VIDEO PRODUCTION', 'PRINT ADVERTISING', 'SECURITY',
       'COMPUTER SOFTWARE', 'ON-LINE ADVERTISING',
       'PUBLICATIONS/ SUBSCRIPTIONS', 'POLLING', 'TECHNOLOGY SERVICES',
       'EVENT PLANNING SERVICES', 'TEXT MESSAGES', 'PHOTOGRAP

In [9]:
len(df_obama2)

84116

In [10]:
catsum = df_obama2.groupby('disbursement_description').agg({'disbursement_description':'count','disbursement_amount':'sum'})

In [11]:
catsum.rename(columns={'disbursement_description':'count'}, inplace=True)

In [12]:
catsum.sort_values('count',ascending=False,na_position='first')

,count,disbursement_amount
disbursement_description,,
TRAVEL/LODGING,29867,2.272525e+07
PAYROLL,16156,3.064731e+07
TRAVEL AGENCY FEES,9805,1.771490e+05
TELEPHONE,3523,2.342954e+06
PRINTING,3264,1.072391e+07
...,...,...
EQUIPMENT IN-KIND,1,2.600000e+02
DEPOSIT ON ACCOUNT,1,-4.189585e+06
DECORATIONS IN KIND,1,2.597000e+02


In [13]:
# many uncategorized items under 'other'
df_obama2.groupby('disbursement_purpose_category').agg({'disbursement_purpose_category':'count','disbursement_amount':'sum'})

,disbursement_purpose_category,disbursement_amount
disbursement_purpose_category,,
ADMINISTRATIVE,4286,2.577538e+07
ADVERTISING,4729,4.190138e+08
FUNDRAISING,1428,1.454805e+06
MATERIALS,3,1.915320e+03
OTHER,43802,2.301702e+08
TRAVEL,29868,2.272535e+07


In [3]:
df_obama2.disbursement_amount.sum()

699141504.0099999

## <div class="alert alert-block alert-success"><center>Text Clustering - Obama Part<center>

In [14]:
# Get token list "lda_tokens"
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text): 
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace(): #skip space
            continue
        elif token.like_num: # delete numbers
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

# Get the puntuations set
import string
en_punct = set(string.punctuation)

# Get the stop words set
import nltk
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

# Get the tokens for LDA analysis (function)
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

def get_lemma(word):
    return WordNetLemmatizer().lemmatize(word) 

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if token not in en_punct] # delete punctuations
    tokens = [token for token in tokens if len(token) > 1] # at least 2 letters consist a word
    tokens = [token for token in tokens if token not in en_stop] # delete stop words
    tokens = [get_lemma(token) for token in tokens] # delete word variants for accurate result
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhanghuishan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhanghuishan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import random
text_data = []
for line in df_obama2['disbursement_description']:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        #print(tokens)
        text_data.append(tokens)

In [16]:
# Cluster topics
from gensim import corpora 
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [25]:
import gensim # package for topic modeling
NUM_TOPICS = 6 # trial number based on understanding of the data file & research aims
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=20)
ldamodel.save('model.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.311*"fee" + 0.308*"travel" + 0.307*"agency" + 0.011*"parking"')
(1, '0.194*"computer" + 0.169*"chargeback" + 0.169*"equipment" + 0.068*"sound"')
(2, '0.748*"payroll" + 0.074*"tax" + 0.033*"planning" + 0.029*"service"')
(3, '0.250*"printing" + 0.101*"medium" + 0.086*"fee" + 0.079*"production"')
(4, '0.425*"lodging" + 0.425*"travel" + 0.044*"telephone" + 0.020*"event"')
(5, '0.206*"supply" + 0.191*"office" + 0.108*"catering" + 0.108*"facility"')


In [26]:
import pyLDAvis.gensim 
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model.gensim')
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display) 

## <div class="alert alert-block alert-success"><center>Data Preparation - Romney Part<center>

In [34]:
df_romney=pd.read_csv('romney2012.csv')

/Users/zhanghuishan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5,10,11,14,16,17,19,23,24,26,27,33,34,44,45,47,48,49,50,51,66,72,73) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [35]:
len(df_romney)

77246

In [36]:
df_romney = df_romney[['committee_id', 'report_year', 'disbursement_date','recipient_name',\
                     'recipient_state','recipient_city',\
                     'disbursement_purpose_category','disbursement_description','disbursement_amount']]

In [37]:
df_romney.drop(df_romney[df_romney['disbursement_purpose_category'] == 'REFUNDS'].index, inplace = True) 
df_romney.drop(df_romney[df_romney['disbursement_description'] == 'CONTRIBUTION REFUND'].index, inplace = True) 
df_romney = df_romney[df_romney['disbursement_description'].notna()]

In [38]:
df_romney.groupby('disbursement_purpose_category').agg({'disbursement_purpose_category':'count','disbursement_amount':'sum'})

,disbursement_purpose_category,disbursement_amount
disbursement_purpose_category,,
ADMINISTRATIVE,9834,3.419669e+07
ADVERTISING,876,2.033062e+08
FUNDRAISING,130,4.059917e+07
LOAN-REPAYMENTS,11,1.880000e+07
MATERIALS,1439,6.084943e+06
OTHER,50702,3.066074e+08
TRANSFERS,116,3.130808e+08
TRAVEL,10528,6.969824e+06


In [39]:
df_romney.groupby('disbursement_description').agg({'disbursement_description':'count','disbursement_amount':'sum'})

,disbursement_description,disbursement_amount
disbursement_description,,
AIR,12048,2.108534e+07
AIR (6/7/12),1,9.194250e+03
AIR (05/30/12),1,2.478680e+03
AIR (06/20/2012),1,1.779013e+04
AIR (07/13/12),1,1.677419e+04
...,...,...
VOIDED CHECK - ORIGINAL DATE: 09/24/2012,1,-2.885000e+03
VOIDED CHECK - ORIGINAL DATE: 10/16/2012,1,-3.374400e+03
WEB DEVELOPMENT,1,3.990000e+02


In [40]:
len(df_romney)

73636

## <div class="alert alert-block alert-success"><center>Text Clustering - Romney Part<center>

In [41]:
# Import data for LDA analysis and return text_data
import random
text_data = []
for line in df_romney['disbursement_description']:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        #print(tokens)
        text_data.append(tokens)

In [42]:
from gensim import corpora 
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [63]:
import gensim # package for topic modeling
NUM_TOPICS = 4 # trial number based on understanding of the data file & research aims
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=20)
ldamodel.save('model.gensim')
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.379*"payroll" + 0.223*"service" + 0.058*"catering" + 0.035*"printing" + 0.035*"design"')
(1, '0.459*"air" + 0.298*"lodging" + 0.065*"credit" + 0.027*"purchase" + 0.018*"office"')
(2, '0.121*"travel" + 0.121*"food" + 0.098*"ground" + 0.098*"transportation" + 0.092*"consulting"')
(3, '0.318*"fee" + 0.302*"bank" + 0.100*"rental" + 0.088*"car" + 0.027*"expense"')


In [64]:
import pyLDAvis.gensim 
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model.gensim')
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display) 